## Crypto currency database


The database we will use contains two tables `cryptocoins_cryptocurrency` and `cryptocoins_exchange`.

### **Instructions**    

Using the given sqlite3 connection and the `read_sql` method write a single query that:

* Join both tables and return the list of cryptocurrencies with its exchanges. You should use the column with the exchange ID on each table to perform the join.

* As both tables have a `name` column you should rename `cryptocoins_cryptocurrency.name` as `coin_name` and `cryptocoins_exchange.name` as `exchange`. Also select `symbol`, `price_usd` and `percent_change_7d` columns.

* Store the information in a `crypto_df` DataFrame.

* Once you have the `crypto_df` DataFrame, create a new `weekly_change_df` with the `crypto_df` data sorted by `percent_change_7d` from highest to lowest.

In [35]:
import pandas as pd
import sqlite3

In [36]:
conn = sqlite3.connect(':memory:')

In [37]:
c = conn.cursor()

In [38]:
c.executescript(open('files/cryptos.sql', 'r').read())

First we will check the data from bouth tables `cryptocoins_cryptocurrency` and `cryptocoins_exchange` to understand the data we are working with. 

In [39]:
cryptocoins_cryptocurrency_table = pd.read_sql('SELECT * FROM cryptocoins_cryptocurrency', conn)

In [40]:
cryptocoins_cryptocurrency_table.head()

,id,name,slug,symbol,rank,price_usd,price_btc,volume_usd_24h,market_cap_usd,available_supply,total_supply,max_supply,percent_change_1h,percent_change_24h,percent_change_7d,exchange_id,last_updated
0,1,Bitcoin,bitcoin,BTC,1,8707.37,1.000000,1.874329e+10,157139896329,18046762,18046762,2.100000e+07,-0.12,-1.08,-5.91,1,2019-11-14 13:02:23.293956
1,2,Ethereum,ethereum,ETH,2,186.50,0.021433,7.703154e+09,20244944406,108549236,108549236,NaN,-0.21,-0.33,-0.66,1,2019-11-14 13:02:23.326936
2,3,XRP,ripple,XRP,3,0.27,0.000031,1.320452e+09,11621563439,43298481757,99991298961,1.000000e+11,-0.33,-1.83,-8.22,1,2019-11-14 13:02:23.337368
3,4,Bitcoin Cash,bitcoin-cash,BCH,4,278.92,0.032053,1.759249e+09,5051906817,18112350,18112350,2.100000e+07,0.17,-3.00,-4.76,3,2019-11-14 13:02:23.348683
4,5,Tether,tether,USDT,5,1.01,0.000116,2.172326e+10,4141681919,4108044456,4207771504,NaN,-0.23,0.33,0.45,1,2019-11-14 13:02:23.360212


In [41]:
cryptocoins_exchange_table = pd.read_sql('SELECT * FROM cryptocoins_exchange', conn)

In [42]:
cryptocoins_exchange_table.head()

,id,name,url
0,1,Bitstamp,https://www.bitstamp.net
1,2,Poloniex,https://poloniex.com
2,3,Binance,https://www.binance.com/
3,4,OKEx,https://www.okex.com


Now let´s join both tables and create the `crypto_df` DataFrame. We can achieve this with `INNER JOIN` or `LEFT JOIN` as we want to get all the data from both tables. We will use `INNER JOIN` in this case. We just need to 'connect' the two tables by using the foreign key `exchange_id` from the `cryptocoins_cryptocurrency` table and the primary key `id` from the `cryptocoins_exchange` table, to made the join between them.

In [43]:
crypto_df = pd.read_sql(
    "SELECT cryptocoins_cryptocurrency.id, cryptocoins_cryptocurrency.name AS coin_name, symbol, price_usd, percent_change_7d, cryptocoins_exchange.name AS exchange, cryptocoins_exchange.url AS exchange_url FROM cryptocoins_cryptocurrency INNER JOIN cryptocoins_exchange ON cryptocoins_cryptocurrency.exchange_id = cryptocoins_exchange.id;",
    conn,
).set_index('id')

In [44]:
crypto_df.head()

,coin_name,symbol,price_usd,percent_change_7d,exchange,exchange_url
id,,,,,,
1,Bitcoin,BTC,8707.37,-5.91,Bitstamp,https://www.bitstamp.net
2,Ethereum,ETH,186.50,-0.66,Bitstamp,https://www.bitstamp.net
3,XRP,XRP,0.27,-8.22,Bitstamp,https://www.bitstamp.net
4,Bitcoin Cash,BCH,278.92,-4.76,Binance,https://www.binance.com/
5,Tether,USDT,1.01,0.45,Bitstamp,https://www.bitstamp.net


Now let´s create a new DataFrame called `weekly_change_df` with the `crypto_df` data sorted by `percent_change_7d` from highest to lowest. We can achieve this with the `sort_values` method.

In [45]:
crypto_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, 1 to 100
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   coin_name          100 non-null    object 
 1   symbol             100 non-null    object 
 2   price_usd          100 non-null    float64
 3   percent_change_7d  100 non-null    float64
 4   exchange           100 non-null    object 
 5   exchange_url       100 non-null    object 
dtypes: float64(2), object(4)
memory usage: 5.5+ KB


In [46]:
weekly_change_df = crypto_df.sort_values('percent_change_7d', ascending=False)

In [47]:
weekly_change_df # top 5 coins with the highest weekly change

,coin_name,symbol,price_usd,percent_change_7d,exchange,exchange_url
id,,,,,,
66,DxChain Token,DX,0.00,89.14,OKEx,https://www.okex.com
40,MINDOL,MIN,1.36,72.90,Bitstamp,https://www.bitstamp.net
75,Seele,SEELE,0.11,39.59,Bitstamp,https://www.bitstamp.net
62,Komodo,KMD,0.86,22.75,Poloniex,https://poloniex.com
27,VeChain,VET,0.01,21.49,OKEx,https://www.okex.com
...,...,...,...,...,...,...
36,Insight Chain,INB,0.64,-13.37,Bitstamp,https://www.bitstamp.net
58,Bytom,BTM,0.11,-13.42,OKEx,https://www.okex.com
65,Silverway,SLV,0.95,-13.84,Poloniex,https://poloniex.com


In [48]:
c.close()
conn.close()